In [1]:
import os 
os.environ['SPARK_HOME']=r'C:/spark/'
os.environ['HADOOP_HOME'] = r'C:/hadoop/'
os.environ['PYSPARK_DRIVER_PYTHON']='jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='lab'
os.environ['PYSPARK_PYTHON']='python'

**Spark datasets**

1. RDD
- Low Level API
- Funcionales

2. DataFrames
- High Level API
- Relacional
- Optimizados para querys

**Mapper Transformations**

1. map(f) >> Relacion 1 a 1 >> aplica la funcion f() a un RDD
2. mapValues(f) >> Relacion 1 a 1 >> pasa cada pareja (key,value) del RDD a f()
3. flatMap(f) >> Relacion 1 a muchos >> Aplica la funcion f() a los elementos del RDD y aplana los resultados
4. flatMapValues(f) >> Relacion 1 a muchos >> Pasa cada valor (key,value) del RDD por el flatMap(f) sin cambiar keys
5. mapPartitions(f) >> Relacion muchos a 1 >> Devuelve un RDD aplicando la funcion f() a cada particion del RDD fuente 

# 1. map()

- Es la más común y se puede aplicar a RDD o Dataframes
- Se aplica la ufncion a cada elemento, no es secuencial 
- El source RDD se particiona para ser procesado independiente y concurrente (e.g si tu dataset tiene 40 billones de elementos se puede particionar en 2 millones de elementos aunque dependera de tu cluster)

In [2]:
def map_david(x):
    if (x> 1):
        return x+3
    else:
        return 0

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("App1").getOrCreate()
# Data
data = [1, 3, -1, 2, -2, 4, 5, 6]
# Creamos el RDD
rdd = spark.sparkContext.parallelize(data) # RDD[integer]
rdd.collect()

[1, 3, -1, 2, -2, 4, 5, 6]

In [5]:
spark

In [6]:
# Lambda expression
rdd2 = rdd.map(lambda x: map_david(x)) # RDD [Integer]
rdd2.collect()

[0, 6, 0, 5, 0, 7, 8, 9]

In [7]:
# USando la funcion
rdd3 = rdd.map(map_david) # RDD Ìnteger]
rdd3.collect()

[0, 6, 0, 5, 0, 7, 8, 9]

In [8]:
# Otra forma
rdd4 = rdd.map(lambda x: (x, map_david(x))) # RDD [Integer]
rdd4.collect()

[(1, 0), (3, 6), (-1, 0), (2, 5), (-2, 0), (4, 7), (5, 8), (6, 9)]

In [9]:
spark.stop()

In [10]:
# Otro ejemplo
spark = SparkSession.builder.appName("App2").getOrCreate()
par= [('David',2),('Juan',-1),('Andrea',5),('Pedro',-3),('Sofia',8)]
# Este es un RDD [(String, Integer)]
rdd = spark.sparkContext.parallelize(par)
rdd.collect()

[('David', 2), ('Juan', -1), ('Andrea', 5), ('Pedro', -3), ('Sofia', 8)]

In [12]:
rdd2 = rdd.map(lambda kv: (kv[0], kv[1], kv[1] + 100))
# Esto ahora es un RDD[(String, Integer, Integer)]
rdd2.collect()

[('David', 2, 102),
 ('Juan', -1, 99),
 ('Andrea', 5, 105),
 ('Pedro', -3, 97),
 ('Sofia', 8, 108)]

In [13]:
# Similar a si hicieramos esto
def crear_parejas(string):
    tokens= string.split(",")
    return (tokens[0],tokens[1],tokens[2])
palabras=['David,10,11','Juan,5,4','Pedro,4,5','Sofia,5,7']
rdd= spark.sparkContext.parallelize(palabras)
rdd.collect()

['David,10,11', 'Juan,5,4', 'Pedro,4,5', 'Sofia,5,7']

In [14]:
patrones= rdd.map(crear_parejas) # RDD[(String,String,String)]
patrones.collect()

[('David', '10', '11'),
 ('Juan', '5', '4'),
 ('Pedro', '4', '5'),
 ('Sofia', '5', '7')]

In [16]:
# Usemos datos reales
def parsing_david(registro):
    tokens= registro.split(",")
    # Extraer valores relevantes
    edad= int(tokens[2])
    amigos= int(tokens[3])
    return (edad, amigos)
# Definir path
path= 'usuarios.txt'
app= spark.sparkContext.textFile(path)
patrones= app.map(parsing_david)
patrones.collect()

[(23, 130),
 (21, 120),
 (25, 110),
 (24, 170),
 (25, 180),
 (28, 190),
 (27, 170),
 (21, 110),
 (26, 130),
 (27, 190)]

In [24]:
# Algunas operaciones adicionales
patrones.mapValues(lambda x: (x,1)).collect()

[(23, (130, 1)),
 (21, (120, 1)),
 (25, (110, 1)),
 (24, (170, 1)),
 (25, (180, 1)),
 (28, (190, 1)),
 (27, (170, 1)),
 (21, (110, 1)),
 (26, (130, 1)),
 (27, (190, 1))]

In [26]:
# Como algunas edades se repiten podemos contar entonces
# RDD[(Integer, (Integer,Integer))]
totales=patrones.\
    mapValues(lambda x: (x,1)).\
    reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
# Estamos adicionando las tuplas en este caso y agrupando por el primer Integer
totales.collect()

[(24, (170, 1)),
 (28, (190, 1)),
 (26, (130, 1)),
 (23, (130, 1)),
 (21, (230, 2)),
 (25, (290, 2)),
 (27, (360, 2))]

In [27]:
# Ahora promedios 
totales.mapValues(lambda x: float(x[0]/float(x[1]))).collect()

[(24, 170.0),
 (28, 190.0),
 (26, 130.0),
 (23, 130.0),
 (21, 115.0),
 (25, 145.0),
 (27, 180.0)]

In [29]:
# Sobre dataframes
# No tienen formalmente la funcion map() pero se puede alcanzar el mismo resultado
# de diversas formas con wothColumn() o .drop()
tuples3 = [ ('david', 440, 'PHD'), ('juan', 420, 'PHD'),
           ('bob', 280, 'MS'), ('betsy', 200, 'MS'),
           ('andrea', 180, 'BS'), ('maria', 100, 'BS') ]
df = spark.createDataFrame(tuples3, ["nombre", "monto", "educacion"])
df.show()

+------+-----+---------+
|nombre|monto|educacion|
+------+-----+---------+
| david|  440|      PHD|
|  juan|  420|      PHD|
|   bob|  280|       MS|
| betsy|  200|       MS|
|andrea|  180|       BS|
| maria|  100|       BS|
+------+-----+---------+



In [31]:
# Supongamos que tenemos que calcular el bono a pagar
# Corresponde al 10% del salario 
df.rdd.\
    map(lambda x: (x['nombre'],x['monto'],
                   x['educacion'],int(x['monto'])/10)).\
    toDF(['nombre','monto','educacion','bono']).show()

+------+-----+---------+----+
|nombre|monto|educacion|bono|
+------+-----+---------+----+
| david|  440|      PHD|44.0|
|  juan|  420|      PHD|42.0|
|   bob|  280|       MS|28.0|
| betsy|  200|       MS|20.0|
|andrea|  180|       BS|18.0|
| maria|  100|       BS|10.0|
+------+-----+---------+----+



In [33]:
# Si hay muchas columnas que pasa? 
df.rdd.\
    map(lambda x: x +\
        (str(x['monto']/10),)).\
    toDF(df.columns +['bono']).show()

+------+-----+---------+----+
|nombre|monto|educacion|bono|
+------+-----+---------+----+
| david|  440|      PHD|44.0|
|  juan|  420|      PHD|42.0|
|   bob|  280|       MS|28.0|
| betsy|  200|       MS|20.0|
|andrea|  180|       BS|18.0|
| maria|  100|       BS|10.0|
+------+-----+---------+----+



In [39]:
import pandas as pd 
p=pd.DataFrame(tuples3,columns=['nombre','monto','educacion'])
p.apply(lambda row: tuple(list(row) + [str(row['monto'] / 10)]), axis=1)

0    (david, 440, PHD, 44.0)
1     (juan, 420, PHD, 42.0)
2       (bob, 280, MS, 28.0)
3     (betsy, 200, MS, 20.0)
4    (andrea, 180, BS, 18.0)
5     (maria, 100, BS, 10.0)
dtype: object

In [43]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

# Crear la spark session
spark = SparkSession.builder.appName("App").getOrCreate()
# Aplicando el WithColumn
df.withColumn("bono", lit(df["monto"] / 10)).show()

+------+-----+---------+----+
|nombre|monto|educacion|bono|
+------+-----+---------+----+
| david|  440|      PHD|44.0|
|  juan|  420|      PHD|42.0|
|   bob|  280|       MS|28.0|
| betsy|  200|       MS|20.0|
|andrea|  180|       BS|18.0|
| maria|  100|       BS|10.0|
+------+-----+---------+----+

